## 매 시도마다 돌릴 것들

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets --quiet

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

data_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/'  # 이미지 및 텍스트 파일 폴더

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
from tqdm import tqdm
from collections import Counter
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoConfig
from scipy.special import softmax
import shutil
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import seaborn as sns
import torch.nn as nn
from sklearn.model_selection import train_test_split

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## 전처리

In [2]:
# 텍스트 데이터 로드
texts_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/texts.csv', encoding='unicode_escape')
label_txt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/label.txt', sep='\t', header=0)
label_txt.columns = ['ID', 'Annotator1', 'Annotator2', 'Annotator3']
label_jpg = label_txt.copy()

def pre_text(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

texts_df['Text'] = texts_df['Text'].apply(pre_text)

In [3]:
dic = {'positive': 1, 'neutral':0, 'negative': -1}
def pre_labeltext(text):
    new_text = text.split(",")[0]
    return dic[new_text]
label_txt['Annotator1'] = label_txt['Annotator1'].apply(pre_labeltext)
label_txt['Annotator2'] = label_txt['Annotator2'].apply(pre_labeltext)
label_txt['Annotator3'] = label_txt['Annotator3'].apply(pre_labeltext)
label_txt['label'] = label_txt['Annotator1'] + label_txt['Annotator2'] + label_txt['Annotator3']
label_txt['label'] = label_txt['label']/3
label_txt = label_txt.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [4]:
def pre_labeltext(text):
    new_text = text.split(",")[1]
    return dic[new_text]
label_jpg['Annotator1'] = label_jpg['Annotator1'].apply(pre_labeltext)
label_jpg['Annotator2'] = label_jpg['Annotator2'].apply(pre_labeltext)
label_jpg['Annotator3'] = label_jpg['Annotator3'].apply(pre_labeltext)
label_jpg['label'] = label_jpg['Annotator1'] + label_jpg['Annotator2'] + label_jpg['Annotator3']
label_jpg['label'] = label_jpg['label']/3
label_jpg = label_jpg.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [5]:
# 라벨 데이터와 텍스트 데이터 병합
merged_text = pd.merge(label_txt, texts_df, on='ID')

## 데이터 라벨링-모델 일치율 확인(만족스럽다)

In [6]:
def checksentiment(listtexts, listlabels):
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    batch_size = 16
    sentiment_scores = []

    for i in tqdm(range(0, len(listtexts), batch_size)):
        batch_texts = listtexts[i:i+batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
        encoded_input.to(device)

        with torch.no_grad():
            outputs = model(**encoded_input)

        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        scores = probabilities.cpu().numpy()

        pos_probs = scores[:, 2]  # 긍정 확률
        neg_probs = scores[:, 0]  # 부정 확률
        sentiments = pos_probs - neg_probs  # 감성 점수
        sentiment_scores.extend(sentiments)

    listlabels = np.array(listlabels)
    sentiment_scores = np.array(sentiment_scores)

    # L2 Loss
    l2_loss = np.mean((sentiment_scores - listlabels) ** 2)
    print(f"L2 Loss: {l2_loss}")

    return sentiment_scores

In [7]:
lab = merged_text['label'].tolist()
sentiment_scores = checksentiment(merged_text['Text'].tolist(), lab)
check = 0
for i in range(len(sentiment_scores)):
    if sentiment_scores[i] * lab[i] < 0:
        check+=1
check

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

100%|██████████| 1225/1225 [00:20<00:00, 59.79it/s]

L2 Loss: 0.22856436414728568


1713

## 이미지 설명-감성 분석 점수 확인(학습 전 저성능 확인)

In [8]:
#자기 전에 켜 둘 것
source_dir = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data'

target_dir = '/content/data'

# 대상 폴더가 없으면 생성
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

loader = label_txt['ID'].tolist()

for filename in tqdm(loader):
    filename_with_ext = f"{filename}.jpg"
    src_file = os.path.join(source_dir, filename_with_ext)
    dst_file = os.path.join(target_dir, filename_with_ext)

    try:
        shutil.copy2(src_file, dst_file)
    except Exception as e:
        print(f"파일 복사 중 오류 발생: {filename_with_ext}, 오류: {e}")

100%|██████████| 19598/19598 [1:49:18<00:00,  2.99it/s]


In [9]:
merged = pd.merge(merged_text, label_jpg, on='ID')

In [10]:
merged = pd.merge(merged, label_txt, on='ID')

In [11]:
scores = np.array(sentiment_scores)
sim = merged['label_x'] * merged['label_y'] > 0
merged['pred'] = scores
input_df = merged[sim]

In [12]:
input_df.drop([1336], axis=0, inplace=True)

<ipython-input-12-a5e798a59b6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df.drop([1336], axis=0, inplace=True)


In [14]:
image_dir = '/content/data'
def jpg(num):
    return f"{num}.jpg"
image_ids = input_df['ID'].apply(jpg).tolist()
actual_labels = input_df['pred'].tolist()

## 멀티모달 이미지 감성-설명 학습

In [15]:
labels = actual_labels
image_dir = '/content/data'

In [24]:
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model.to(device)

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-23): 24 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (projection): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1024,),

In [17]:
train_image_ids, val_image_ids, train_labels, val_labels = train_test_split(image_ids, actual_labels, test_size=0.2, random_state=529)

In [18]:
# 감성 분석 모델 로드 (고정된 상태로 사용)
MODEL_SA = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer_sa = AutoTokenizer.from_pretrained(MODEL_SA)
config_sa = AutoConfig.from_pretrained(MODEL_SA)
model_sa = AutoModelForSequenceClassification.from_pretrained(MODEL_SA)
model_sa.to(device)
model_sa.eval()  # 평가 모드

# 감성 분석 모델의 파라미터를 고정
for param in model_sa.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
# 손실 함수 정의 (예: MSELoss)
criterion = nn.MSELoss()

# 옵티마이저 정의 (BLIP 모델의 파라미터만 업데이트)
optimizer = torch.optim.AdamW(blip_model.parameters(), lr=1e-4)

In [20]:
num_epochs = 100  # 최대 에포크 수를 크게 설정
patience = 5      # 검증 손실이 개선되지 않는 에포크 수 허용치
best_val_loss = float('inf')
epochs_no_improve = 0
early_stop = False

In [21]:
blip_model.eval()
val_loss = 0.0
with torch.no_grad():
    for i in tqdm(range(len(val_image_ids))):
        pix = Image.open(os.path.join(image_dir, val_image_ids[i])).convert('RGB')
        pix = processor(pix, return_tensors='pt')
        pix = {k: v.to(device) for k, v in pix.items()}
        labels = val_labels[i]  # [batch_size]
        outputs = blip_model.generate(**pix)
        captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]

        inputs_sa = tokenizer_sa(captions, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs_sa = model_sa(**inputs_sa)
        scores = outputs_sa.logits
        scores = scores.cpu().numpy()
        probs = softmax(scores, axis=1)
        pos_probs = probs[:, 2]
        neg_probs = probs[:, 0]
        sentiment_scores = pos_probs - neg_probs
        sentiment_scores = torch.tensor(sentiment_scores, dtype=torch.float).to(device)

        loss = abs(sentiment_scores - labels)
        val_loss += loss.item()

avg_val_loss = val_loss / len(val_image_ids)
print(f"Validation Loss: {avg_val_loss:.4f}")

  0%|          | 0/2569 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 2569/2569 [14:06<00:00,  3.03it/s]

Validation Loss: 0.6028


In [ ]:
for epoch in range(num_epochs):
    blip_model.train()
    total_loss = 0.0
    for i in tqdm(range(len(train_image_ids))):
        pix = Image.open(os.path.join(image_dir, train_image_ids[i])).convert('RGB')
        pix = processor(pix, return_tensors='pt')
        pix = {k: v.to(device) for k, v in pix.items()}
        labels = train_labels[i]  # [batch_size]
        # BLIP 모델을 통해 캡션 생성
        outputs = blip_model.generate(**pix)
        # 생성된 토큰 ID를 디코딩하여 텍스트로 변환
        captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]
        blip_model.train()
        # 감성 분석 모델을 통해 감성 점수 계산
        inputs_sa = tokenizer_sa(captions, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs_sa = model_sa(**inputs_sa)
        scores = outputs_sa.logits  # [batch_size, num_labels]

        # 감성 점수 계산 (pos_probs - neg_probs)
        scores = scores.cpu().numpy()
        probs = softmax(scores, axis=1)
        pos_probs = probs[:, 2]  # 긍정 확률
        neg_probs = probs[:, 0]  # 부정 확률
        sentiment_scores = pos_probs - neg_probs  # [batch_size]

        # 감성 점수를 텐서로 변환
        sentiment_scores = torch.tensor(sentiment_scores, dtype=torch.float).to(device)

        # 손실 계산 (예측된 감성 점수와 실제 라벨 간의 차이)
        loss = abs(sentiment_scores - labels)
        loss.requires_grad_(True)
        # 역전파 및 옵티마이저 스텝
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_image_ids)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 검증 루프
    blip_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i in tqdm(range(len(val_image_ids))):
            pix = Image.open(os.path.join(image_dir, val_image_ids[i])).convert('RGB')
            pix = processor(pix, return_tensors='pt')
            pix = {k: v.to(device) for k, v in pix.items()}
            labels = val_labels[i]  # [batch_size]
            outputs = blip_model.generate(**pix)
            captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]

            inputs_sa = tokenizer_sa(captions, return_tensors="pt", truncation=True, padding=True).to(device)
            outputs_sa = model_sa(**inputs_sa)
            scores = outputs_sa.logits
            scores = scores.cpu().numpy()
            probs = softmax(scores, axis=1)
            pos_probs = probs[:, 2]
            neg_probs = probs[:, 0]
            sentiment_scores = pos_probs - neg_probs
            sentiment_scores = torch.tensor(sentiment_scores, dtype=torch.float).to(device)

            loss = abs(sentiment_scores - labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_image_ids)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    # Early Stopping 체크
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        # 최적의 모델 저장 (옵션)
        torch.save(blip_model.state_dict(), 'best_blip_model.pth')
        print("Validation loss improved. Model saved.")
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation loss for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break


  0%|          | 0/11557 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 11557/11557 [1:02:50<00:00,  3.07it/s]


Epoch [1/100], Loss: 2.3986


  0%|          | 0/1285 [00:00<?, ?it/s]


AttributeError: 'float' object has no attribute 'size'

In [22]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, image_ids, labels, image_dir, processor):
        self.image_ids = image_ids
        self.labels = labels
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_ids[idx])
        image = Image.open(image_path).convert('RGB')
        # 이미지 전처리 수행
        pix = self.processor(image, return_tensors='pt')
        # 배치 차원 제거 (processor는 보통 배치 차원을 포함하여 반환)
        pix = {k: v.squeeze(0) for k, v in pix.items()}
        label = self.labels[idx]
        return pix, label

# 데이터셋과 데이터로더 생성
train_dataset = ImageDataset(train_image_ids, train_labels, image_dir, processor)
val_dataset = ImageDataset(val_image_ids, val_labels, image_dir, processor)

batch_size = 32  # 적절한 배치 사이즈 설정
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
for epoch in range(num_epochs):
    blip_model.train()
    total_loss = 0.0
    for batch in tqdm(train_dataloader):
        pix, labels = batch
        pix = {k: v.to(device) for k, v in pix.items()}
        labels = labels.to(device).float()

        # BLIP 모델을 통해 캡션 생성
        outputs = blip_model.generate(**pix)
        # 생성된 토큰 ID를 디코딩하여 텍스트로 변환
        captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]

        # 감성 분석 모델을 통해 감성 점수 계산
        inputs_sa = tokenizer_sa(captions, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs_sa = model_sa(**inputs_sa)
        scores = outputs_sa.logits.cpu().numpy()
        probs = softmax(scores, axis=1)
        pos_probs = probs[:, 2]  # 긍정 확률 (모델에 따라 인덱스 확인 필요)
        neg_probs = probs[:, 0]  # 부정 확률
        sentiment_scores = pos_probs - neg_probs  # [batch_size]

        # 감성 점수를 텐서로 변환
        sentiment_scores = torch.from_numpy(sentiment_scores).float().to(device)

        # 손실 계산 (예: MSELoss 사용)
        loss = criterion(sentiment_scores, labels)
        loss.requires_grad_(True)
        # 역전파 및 옵티마이저 스텝
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")

    # 검증 루프
    blip_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch1 in tqdm(val_dataloader):
            pix1, labels1 = batch1
            pix1 = {k: v.to(device) for k, v in pix1.items()}
            labels1 = labels1.to(device).float()

            outputs1 = blip_model.generate(**pix1)
            captions1 = [processor.decode(output, skip_special_tokens=True) for output in outputs1]

            inputs_sa1 = tokenizer_sa(captions1, return_tensors="pt", truncation=True, padding=True).to(device)
            outputs_sa1 = model_sa(**inputs_sa1)
            scores1 = outputs_sa1.logits.cpu().numpy()
            probs1 = softmax(scores1, axis=1)
            pos_probs1 = probs1[:, 2]
            neg_probs1 = probs1[:, 0]
            sentiment_scores1 = pos_probs1 - neg_probs1
            sentiment_scores1 = torch.tensor(sentiment_scores1, dtype=torch.float).to(device)

            loss = criterion(sentiment_scores1, labels1)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss}")

    # 조기 종료 체크 및 모델 저장
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(blip_model.state_dict(), 'best_blip_model.pth')
        print("Validation loss improved. Model saved.")
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation loss for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break


  0%|          | 0/322 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 25%|██▌       | 82/322 [06:03<17:42,  4.43s/it]

In [29]:
len(labels)

32

In [26]:
blip_model.eval()
val_loss = 0.0
with torch.no_grad():
    for batch1 in tqdm(val_dataloader):
        pix1, labels1 = batch1
        pix1 = {k: v.to(device) for k, v in pix1.items()}
        labels1 = labels1.to(device).float()

        outputs1 = blip_model.generate(**pix1)
        captions1 = [processor.decode(output, skip_special_tokens=True) for output in outputs1]

        inputs_sa1 = tokenizer_sa(captions1, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs_sa1 = model_sa(**inputs_sa1)
        scores1 = outputs_sa1.logits.cpu().numpy()
        probs1 = softmax(scores1, axis=1)
        pos_probs1 = probs1[:, 2]
        neg_probs1 = probs1[:, 0]
        sentiment_scores1 = pos_probs1 - neg_probs1
        sentiment_scores1 = torch.tensor(sentiment_scores1, dtype=torch.float).to(device)

        loss = criterion(sentiment_scores1, labels1)
        val_loss += loss.item()

avg_val_loss = val_loss / len(val_dataloader)
print(f"Validation Loss: {avg_val_loss}")

100%|██████████| 81/81 [05:54<00:00,  4.37s/it]

Validation Loss: 0.49966360647001384


차이 없는거보니 추가적으로 뭐 해야할듯... generate 안 쓰는 법 찾아볼까
일단 학습률 조정해보고도 차이 저런지 확인해볼것